Fully Connected Feed-forward Neural Network

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import argparse
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from matplotlib.colors import ListedColormap
import seaborn as sn
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [73]:
def load_graphs(input_dir, class_dict) :

    data, data_labels = [], [] # data containing the graphs and data_labels the associated seizure type labels

    for szr_type in class_dict.keys() :
        szr_label = class_dict[szr_type]
        for _, _, files in os.walk(os.path.join(input_dir,szr_type)) :
            for npy_file in files :
                graph = np.load(os.path.join(input_dir,szr_type,npy_file))

                graph = graph[np.triu_indices(20, k = 1)]

                data.append(graph.flatten()) # graph has to be flattened to be fed to the classifier
                data_labels.append(szr_label)

    return np.array(data), np.array(data_labels)

def train_test_data(input_dir, class_dict) :

    train, train_labels = load_graphs(os.path.join(input_dir,'train'), class_dict)
    test, test_labels = load_graphs(os.path.join(input_dir,'dev'), class_dict)

    return train, test, train_labels, test_labels

In [74]:
input_dir = '../data/v1.5.2/graph_avg_1_5'
szr_types = ['FNSZ','GNSZ']

class_dict = {}
for i, szr_type in enumerate(szr_types) :
    class_dict[szr_type] = i

train, test, train_labels, test_labels = train_test_data(input_dir, class_dict)

In [75]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 200

trainset, testset = [], []
for i in range(len(train)) :
    trainset.append((train[i],train_labels[i]))
for j in range(len(test)) :
    testset.append((test[j],test_labels[j]))
classes = ('FNSZ','GNSZ')

#trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

#testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [61]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(190, 20)
        self.fc2 = nn.Linear(20, 20)
        self.fc3 = nn.Linear(20, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x,dim=0)

In [62]:
FCNN = Net()
FCNN = FCNN.float()
print(FCNN)

Net(
  (fc1): Linear(in_features=190, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=2, bias=True)
)


In [63]:
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(FCNN.parameters(), lr=0.005)

In [76]:
i = 0
X_shapes = []
y_shapes = []
"""
for data in trainloader :  
    X, y = data

    X, y = X.float(), y.type(torch.LongTensor)
    if i == 0 : X_0, y_0 = X.shape, y.shape
    if (X.shape != X_0) or (y.shape != y_0) : print(f'PROBLEM at ({i})',': ',X.shape,', ',y.shape)
    i += 1
"""
for data in trainloader :
    X, y = data
    X, y = X.float(), y.type(torch.LongTensor)
    if X.shape not in X_shapes : 
        X_shapes.append(X.shape)
        print(i)
    if y.shape not in y_shapes : 
        y_shapes.append(y.shape)
    i += 1

# PROBLEM at (19) :  torch.Size([45, 190]) ,  torch.Size([45])
print('X_shapes : ',X_shapes)
print('y_shapes : ',y_shapes)

0
9
X_shapes :  [torch.Size([200, 190]), torch.Size([145, 190])]
y_shapes :  [torch.Size([200]), torch.Size([145])]


In [ ]:
print()

In [66]:
for epoch in range(10): 
    for data in trainloader:  
        X, y = data
        FCNN.zero_grad()
        X, y = X.float(), y.type(torch.LongTensor)
        #X, y = torch.from_numpy(X).float().view(-1,190), torch.Tensor(y)
        output = FCNN(X)
        loss = loss_criterion(output, y)
        loss.backward()
        optimizer.step()  
    print(f"Loss : {round(loss,4)}")


Loss : 0.3150310516357422
Loss : 0.3025282025337219
Loss : 0.276330828666687
Loss : 0.17727479338645935
Loss : 0.3216826915740967
Loss : 0.17098599672317505
Loss : 0.1823856383562088
Loss : 0.11890880763530731
Loss : 0.2640230059623718


RuntimeError: Caught RuntimeError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "C:\Users\rapha\anaconda3\lib\site-packages\torch\utils\data\_utils\worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\rapha\anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "C:\Users\rapha\anaconda3\lib\site-packages\torch\utils\data\_utils\collate.py", line 84, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "C:\Users\rapha\anaconda3\lib\site-packages\torch\utils\data\_utils\collate.py", line 84, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "C:\Users\rapha\anaconda3\lib\site-packages\torch\utils\data\_utils\collate.py", line 64, in default_collate
    return default_collate([torch.as_tensor(b) for b in batch])
  File "C:\Users\rapha\anaconda3\lib\site-packages\torch\utils\data\_util